In [3]:
import pyBigWig

In [4]:
import numpy as np
import gzip
# Open the gzipped file and read the data
with gzip.open('/mnt/lab_data2/anusri/chrombpnet/reference/chr1.uint8.unique.gz', 'rb') as tmp_uMap:
    uMapdata = np.frombuffer(tmp_uMap.read(), dtype=np.uint8)

pos_vec = (uMapdata>0)&(uMapdata<=101)
pos_vec = pos_vec.astype(int)
neg_vec = np.roll(pos_vec, 100)
neg_vec[0:100] = 0

print(set(pos_vec))
print(len(uMapdata))
print(sum(pos_vec))

total_map=pos_vec+neg_vec


{0, 1}
248956422
219161958


In [31]:
pred_bigwig.chroms()['chr1']

248956422

In [32]:
all_val = np.nan_to_num(pred_bigwig.values('chr1', 0, 248956422))

In [33]:
all_val.shape

(248956422,)

In [34]:
set(total_map)

{0, 1, 2}

In [35]:
mappability = [sum(total_map[i:i+100])/200 for i in range(557,248956357, 100)]
total_sig = [sum(all_val[i:i+100]) for i in range(557,248956357, 100)]


In [36]:
len(mappability)

2489558

In [37]:
len(total_sig)

2489558

In [38]:
import pandas as pd

bed_annotate = "/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/auprc_curves/idr_overlap_peak_blacklist_annotated_chr1.bed"

chr1_binned = pd.read_csv(bed_annotate, sep="\t", names=["chr", "start", "end", "idr", "idr_f", "idr_t", "idr_w", "idr_frac", 
                                                         "blacklist", "overlap"])

chr1_binned = chr1_binned[:-1]
print(chr1_binned.shape)
chr1_binned["mappability"] = mappability
chr1_binned["total_sig"] = total_sig
chr1_binned["label"] = 0
chr1_binned.loc[chr1_binned["idr"]==1, "label"] =1
print(sum(chr1_binned["label"]))
#eliminate blacklist
chr1_binned=chr1_binned[chr1_binned["blacklist"]==0]
print(chr1_binned.shape)



(2489558, 10)
74758
(2453566, 13)


In [39]:
chr1_binned.head()

,chr,start,end,idr,idr_f,idr_t,idr_w,idr_frac,blacklist,overlap,mappability,total_sig,label
0,chr1,557,657,0,0,0,100,0.0,0,0,0.0,1.323811,0
1,chr1,657,757,0,0,0,100,0.0,0,0,0.0,1.323811,0
2,chr1,757,857,0,0,0,100,0.0,0,0,0.0,1.323811,0
3,chr1,857,957,0,0,0,100,0.0,0,0,0.0,1.323811,0
4,chr1,957,1057,0,0,0,100,0.0,0,0,0.0,1.323811,0


In [40]:
chr1_binned["overlap"].max()

2

In [41]:
print(sum(chr1_binned["label"]))


74758


In [42]:
chr1_binned["idr"].max()

1

In [43]:
sum(chr1_binned["idr"]==1) # of positives

74758

In [44]:
chr1_binned[chr1_binned["idr"]==1]["idr_frac"].min()

0.5

In [45]:
# eliminate windows with less than 50% overlap with idr
print(chr1_binned.shape)
print(chr1_binned[chr1_binned["idr"]==1].shape)
print(chr1_binned[chr1_binned["idr_frac"]==0.0].shape)
print(sum(chr1_binned["label"]))

chr1_binned = chr1_binned[~((chr1_binned["idr_frac"]>0) & (chr1_binned["idr_frac"]<0.5))]
print(chr1_binned.shape)
print(sum(chr1_binned["label"]))

#chr1_binned[(chr1_binned["idr_frac"]>0) & (chr1_binned["idr"]==0)]


(2453566, 13)
(74758, 13)
(2368887, 13)
74758
(2443645, 13)
74758


In [46]:
sum(chr1_binned["idr"]==1)

74758

In [47]:
# eliminate windows with any overlap with overlap
chr1_binned = chr1_binned[~((chr1_binned["overlap"]==1) & (chr1_binned["idr"]==0))]
print(chr1_binned.shape)

(2418520, 13)


In [48]:
sum(chr1_binned["mappability"]<0.5)

268092

In [49]:
chr1_binned = chr1_binned[chr1_binned["mappability"]>0.5]

In [50]:
chr1_binned.shape

(2149735, 13)

In [51]:
chr1_binned

,chr,start,end,idr,idr_f,idr_t,idr_w,idr_frac,blacklist,overlap,mappability,total_sig,label
98,chr1,10357,10457,0,0,0,100,0.0,0,0,0.915,5.391246,0
99,chr1,10457,10557,0,0,0,100,0.0,0,0,0.790,18.725932,0
100,chr1,10557,10657,0,0,0,100,0.0,0,0,0.705,26.567831,0
101,chr1,10657,10757,0,0,0,100,0.0,0,0,0.605,4.465074,0
102,chr1,10757,10857,0,0,0,100,0.0,0,0,0.565,10.476330,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2489446,chr1,248945157,248945257,0,0,0,100,0.0,0,0,0.705,18.015388,0
2489447,chr1,248945257,248945357,0,0,0,100,0.0,0,0,0.505,13.916912,0
2489450,chr1,248945557,248945657,0,0,0,100,0.0,0,0,0.960,2.872464,0
2489451,chr1,248945657,248945757,0,0,0,100,0.0,0,0,1.000,4.449773,0


In [52]:
import sklearn.metrics

sklearn.metrics.average_precision_score(chr1_binned["label"],chr1_binned["total_sig"])

0.3799409852465915

In [53]:
print(sum(chr1_binned["label"]))


73161


In [54]:
sum(chr1_binned["label"])/chr1_binned.shape[0]

0.0340325668047457